In [1]:
from typing import Any, Callable, Iterable, Iterator, List, Type, Tuple
import json
import time
import functools
import pathlib
import logging

"""
helpers_module.py

Small collection of helper functions and a short usage demo.
Intended to be placed in a new file and imported/used from notebook cells.
"""


logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)


def read_json(path: str | pathlib.Path) -> Any:
    """Read and return JSON from path."""
    p = pathlib.Path(path)
    with p.open("r", encoding="utf-8") as f:
        return json.load(f)


def write_json(path: str | pathlib.Path, data: Any, *, indent: int = 2) -> None:
    """Write JSON data to path (creates parent dirs)."""
    p = pathlib.Path(path)
    p.parent.mkdir(parents=True, exist_ok=True)
    with p.open("w", encoding="utf-8") as f:
        json.dump(data, f, indent=indent, ensure_ascii=False)


def chunked(iterable: Iterable[Any], size: int) -> Iterator[List[Any]]:
    """Yield successive chunks of given size from iterable."""
    if size <= 0:
        raise ValueError("size must be > 0")
    chunk: List[Any] = []
    for item in iterable:
        chunk.append(item)
        if len(chunk) >= size:
            yield chunk
            chunk = []
    if chunk:
        yield chunk


def retry(
    tries: int = 3,
    delay: float = 1.0,
    exceptions: Tuple[Type[BaseException], ...] = (Exception,),
) -> Callable[[Callable[..., Any]], Callable[..., Any]]:
    """
    Decorator to retry a function on exceptions.
    Example:
        @retry(tries=5, delay=0.5)
        def unstable(...): ...
    """
    def decorator(func: Callable[..., Any]) -> Callable[..., Any]:
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            last_exc = None
            for attempt in range(1, tries + 1):
                try:
                    return func(*args, **kwargs)
                except exceptions as exc:
                    last_exc = exc
                    logger.debug("Attempt %d/%d failed: %s", attempt, tries, exc)
                    if attempt < tries:
                        time.sleep(delay)
            # re-raise the last exception
            raise last_exc
        return wrapper
    return decorator


def safe_run(
    func: Callable[..., Any],
    *args,
    default: Any = None,
    on_error: Callable[[BaseException], None] | None = None,
    **kwargs,
) -> Any:
    """
    Run func with args/kwargs, return default on exception.
    Optionally call on_error(exception).
    """
    try:
        return func(*args, **kwargs)
    except Exception as exc:
        if on_error:
            try:
                on_error(exc)
            except Exception:
                logger.exception("on_error handler raised")
        logger.debug("safe_run caught exception: %s", exc)
        return default


class Timer:
    """Context manager for timing a block."""

    def __init__(self, label: str = ""):
        self.label = label
        self.start = 0.0
        self.elapsed = 0.0

    def __enter__(self):
        self.start = time.perf_counter()
        return self

    def __exit__(self, exc_type, exc, tb):
        self.elapsed = time.perf_counter() - self.start
        if self.label:
            logger.info("%s: %.3fs", self.label, self.elapsed)


# Example usage functions that combine helpers


@retry(tries=3, delay=0.5)
def example_task(x: int) -> int:
    """Example task that fails for certain inputs to demonstrate retry."""
    if x % 5 == 0:
        raise ValueError(f"Unstable value: {x}")
    return x * 2


def process_numbers(numbers: Iterable[int], chunk_size: int = 10) -> List[int]:
    """
    Process numbers in chunks. Uses example_task with safe_run to avoid total failure.
    Returns processed results (skips entries that ultimately failed).
    """
    results: List[int] = []
    for chunk in chunked(numbers, chunk_size):
        logger.info("Processing chunk of size %d", len(chunk))
        for n in chunk:
            res = safe_run(example_task, n, default=None, on_error=lambda e: logger.warning("Failed %s -> %s", n, e))
            if res is not None:
                results.append(res)
    return results


if __name__ == "__main__":
    # Simple demo when run as a script
    nums = list(range(1, 31))
    with Timer("process_numbers"):
        out = process_numbers(nums, chunk_size=7)
    print("Processed", len(out), "items. Sample:", out[:10])

INFO:__main__:Processing chunk of size 7
INFO:__main__:Processing chunk of size 7
INFO:__main__:Processing chunk of size 7
INFO:__main__:Processing chunk of size 7
INFO:__main__:Processing chunk of size 2
INFO:__main__:process_numbers: 6.013s


Processed 24 items. Sample: [2, 4, 6, 8, 12, 14, 16, 18, 22, 24]
